# Running DeepSeek R1 locally

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

## Connecting to LM Studio

In [11]:
#@title Connecting to LM Studio
import requests

# Define LM Studio endpoint
LM_STUDIO_URL = "http://127.0.0.1:1234"

# Test that LM Studio is working
try:
    response = requests.get(f"{LM_STUDIO_URL}/v1/models")
    if response.status_code == 200:
        print("✅ LM Studio is running!")
        print("Available models:", response.json())
    else:
        print("⚠️ Unable to reach LM Studio. Make sure it's running.")
except requests.exceptions.ConnectionError:
    print("❌ LM Studio is not running. Start it and try again.")

✅ LM Studio is running!
Available models: {'data': [{'id': 'deepseek-r1-distill-qwen-14b', 'object': 'model', 'owned_by': 'organization_owner'}, {'id': 'text-embedding-nomic-embed-text-v1.5', 'object': 'model', 'owned_by': 'organization_owner'}, {'id': 'deepseek-r1-distill-llama-8b', 'object': 'model', 'owned_by': 'organization_owner'}, {'id': 'deepseek-r1-distill-qwen-7b', 'object': 'model', 'owned_by': 'organization_owner'}], 'object': 'list'}


In [12]:
## Testing out a prompt

In [3]:
#title Testing out a prompt
prompt = "What is the the best fruit? Choose one."

llm_model = "deepseek-r1-distill-qwen-14b"
max_tokens = 4096
temperature = 0.7
# Define the request payload for LM Studio
payload = {
    "model": llm_model,
    "promt": prompt,
    "messages": [
        {"role": "user", "content": prompt}
        ],
    "max_tokens": max_tokens,
    "temperature": temperature
}

# Send the request to LM Studio
response = requests.post(f"{LM_STUDIO_URL}/v1/chat/completions", json=payload)

# Parse and print the response

if response.status_code == 200:
    result = response.json()
    print("🤖 LM Studio:", result["choices"][0]["message"]["content"])  # Fixed key
else:
    print("❌ Something went wrong:", response.text)

🤖 LM Studio: <think>
Okay, so I need to figure out what the best fruit is. Hmm, that's a bit tricky because everyone has different preferences when it comes to fruits. Some people love bananas because they're easy to eat and portable, while others might prefer something like berries for their health benefits or maybe even tropical fruits like mangoes or pineapples if they enjoy something more exotic.

I should consider factors like nutritional value, taste, availability, and versatility. Let's start by thinking about what makes a fruit "good." Usually, fruits are rich in vitamins, minerals, fiber, and antioxidants, which contribute to overall health. Some fruits have unique benefits too, like bananas being good for the digestive system because of their fiber content.

Bananas come to mind as a top choice because they're widely available almost everywhere, easy to eat on the go, and packed with nutrients like potassium, which is great for heart health. Plus, they're versatile in recipes

## Streaming

In [18]:
import requests
import json
import sys

# Define the parameters
prompt = "Explain in 100 words the concept of a black hole."
llm_model = "deepseek-r1-distill-qwen-14b"
max_tokens = 4090
temperature = 0.7

# Define the request payload
payload = {
    "model": llm_model,
    "messages": [
        {"role": "system", "content": "Always answer in 100 words or less. You can ask for more information if needed."},
        {"role": "user", "content": prompt}
    ],
    "max_tokens": max_tokens,
    "temperature": temperature,
    "stream": True
}

# Send the request to LM Studio
with requests.post(f"{LM_STUDIO_URL}/v1/chat/completions", json=payload, stream=True) as response:
    if response.status_code == 200:
        print("🤖 LM Studio (streaming): ", end="", flush=True)
        for line in response.iter_lines():
            line = line.decode("utf-8").strip()
            if not line or line == "data: [DONE]":
                continue  # Skip empty lines and [DONE] signal

            if line.startswith("data: "):
                line = line[6:]  # Remove "data: " prefix

            try:
                data = json.loads(line)
                if "choices" in data and data["choices"]:
                    content = data["choices"][0].get("delta", {}).get("content", "")
                    if content:
                        print(content, end="", flush=True)
                        sys.stdout.flush()  # Force Jupyter to show output
            except json.JSONDecodeError as e:
                print(f"\n❌ JSON Error: {e} - Raw Line: {line}")
                continue  # Skip invalid JSON
    else:
        print("❌ Something went wrong:", response.text)




🤖 LM Studio (streaming): <think>
Okay, so I need to explain what a black hole is in about 100 words. Hmm, where do I start? Well, from what I remember, a black hole is something in space that has really strong gravity. So strong that even light can't escape it, which is why it's called "black" because no light comes out. But wait, how does that work exactly?

I think it starts with a star. When a massive star runs out of fuel, it can't support itself anymore and collapses. This collapse creates something super dense, maybe like a singularity? That sounds right. A singularity is a point where gravity is so intense that spacetime curves infinitely. But I'm not entirely sure about the details.

So, the black hole has this event horizon, which is like the boundary around it. Anything that crosses the event horizon can't escape because the gravitational pull is too strong. Not even light, which means we can't see it directly. That's why they're called black holes—they don't emit any light.
